# Discourse Context Frequency - Historical Sociolinguistics

Paste abstract here.

[https://www.gutenberg.org/cache/epub/1619/pg1619.txt](https://www.gutenberg.org/cache/epub/1619/pg1619.txt)

In [15]:
text = []
with open('data/celestina.txt') as f:
    text = f.read()

In [23]:
import re

Remove unnecessary text at the start. The text begins with the title `[LA CELESTINA]`.

In [30]:
re.search("\[LA CELESTINA\]", text)

<re.Match object; span=(12671, 12685), match='[LA CELESTINA]'>

In [28]:
text_start = text[12671:]
len(text_start)

656375

Remove unnecessary text at the end. The final line of the text is `[A Dios gracias.]`.

In [31]:
re.search("\[A Dios gracias.\]", text_start)

<re.Match object; span=(381463, 381480), match='[A Dios gracias.]'>

In [32]:
new_text = text_start[:381481]
len(new_text)

381481

The text contains many linebreaks `\n`, sometimes multiple in a row. Before beginning, we will replace all instances of `\n` with a single space.

In [415]:
text_2 = re.sub("(\\n)+", " ", new_text)

Remove instances of double spaces.

In [416]:
text_3 = re.sub("\s\s", " ", text_2)

## Identify token words

Move token words into dataframe of structure:

- token
- token frequency in text (log frequency per million)
- preceeding phonological context
- following phonological context
- transmission history (oral, written)
- favorable preceeding (yes, no)
- favorable following (yes, no)
- lexical stress (stress, unstress)
- FFC (proportion of tokens found in favorable)
- obstruent in original Latin (p, f, k)
- hapax token (yes if only instance, no if multiple instances)
- Modern Spanish realization (preservation, palatalization)

The following regex expression will find all words beginning with cl-, fl-, pl- or ll- and will give the surrounding context for each instance.

In [418]:
display(text_3[3735:3780])

' de su perdicion: lleuaronla en alto, no sabe'

In [558]:
words = re.findall(r'.{0,20}\s[p,f,c,l]l\w+.{0,2}', text_3)

In [559]:
len(words)

516

In [560]:
len(set(words))

516

In [562]:
context_ind = [m.span() for m in re.finditer(r'.{0,20}\s[p,f,c,l]l\w+.{0,2}', text_3)]
len(context_ind)

516

In [566]:
context = []
for x in context_ind:
    start = x[0]
    new_start = start - 10
    end = x[1]
    new_end = end + 25
    
    context.append(text_3[new_start:new_end])
len(context)    

516

In [461]:
import pandas as pd

In [579]:
df = pd.DataFrame(words)
df = df.rename(columns={0: "context_small"})
df.head()

,context_small
0,de Milan: mas en los claros i
1,tificio: su fuerte e claro m
2,las de su perdicion: lleuaronla e
3,"ue ella, por ceuo la lleuan:"
4,on es que aplique mi pluma e


In [580]:
df["context_large"] = context

In [581]:
df["word"] = df["context_small"].apply(lambda x: re.search(r'\s[pflc]l\w+', x).group())

In [582]:
df["word"] = df["word"].apply(lambda x: re.sub(r'\s', "", x))
df.head()

,context_small,context_large,word
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma


In [586]:
df["prev"] = df["context_large"].apply(lambda x: re.search(r'.\s[pflc]l', x).group())
df["prev"] = df["prev"].apply(lambda x: re.sub(r'(.)\s[pflc]l', (r"\1"), x))
df.head()

,context_small,context_large,word,prev
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i


In [587]:
import numpy as np

In [589]:
df["prev_fav"] = np.where((df['prev'] == "e") | (df['prev'] == "a") | (df['prev'] == "o"), "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no


In [590]:
df["fol"] = df["context_large"].apply(lambda x: re.search(r'.\s[pflc]l\w', x).group())
df["fol"] = df["fol"].apply(lambda x: re.sub(r'.\s[pflc]l(\w)', (r"\1"), x))
df.head()

,context_small,context_large,word,prev,prev_fav,fol
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u


In [591]:
df["fol_fav"] = np.where((df['fol'] == "e") | (df['fol'] == "a") | (df['fol'] == "o"), "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no


## Find token frequency (ppm) for each token word

In [596]:
import nltk
from nltk.tokenize import RegexpTokenizer

In [601]:
tokenizer = RegexpTokenizer(r'\w+')
total_words = tokenizer.tokenize(text_3)
total_words = len(total_words)

In [605]:
df["count"] = df['word'].apply(lambda x: len(re.findall(x, text_3)))
df["frequency_ppm"] = df["count"].apply(lambda x: (x/total_words)*1000000)
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,frequency,freq_log_ppm,count,frequency_ppm
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040,6,87.710322
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774,23,336.222901
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,29.236774,3.375427,2,29.236774
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,58.473548,4.068574,4,58.473548
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,190.039031,5.247229,13,190.039031


In [604]:
import math
df["freq_log_ppm"] = df["frequency"].apply(lambda x: math.log(x))
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,frequency,freq_log_ppm
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,29.236774,3.375427
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,58.473548,4.068574
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,190.039031,5.247229


## Hapax tokens

In [607]:
df["hapax"] = np.where(df["count"] == 1, "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,frequency,freq_log_ppm,count,frequency_ppm,hapax
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040,6,87.710322,no
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774,23,336.222901,no
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,29.236774,3.375427,2,29.236774,no
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,58.473548,4.068574,4,58.473548,no
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,190.039031,5.247229,13,190.039031,no


## Lexical Stress

In [610]:
from syltippy import syllabize

In [613]:
syllabize("pluma")[1]

0

In [614]:
df["stress"] = np.where(syllabize(df["word"])[1]==0, "stressed", "unstressed")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,frequency,freq_log_ppm,count,frequency_ppm,hapax,stress
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040,6,87.710322,no,stressed
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774,23,336.222901,no,stressed
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,29.236774,3.375427,2,29.236774,no,stressed
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,58.473548,4.068574,4,58.473548,no,stressed
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,190.039031,5.247229,13,190.039031,no,stressed


## FFC: Frequency in favorable environment

In [640]:
values = df.groupby("word")["prev_fav"].value_counts()
values_df = values.to_frame()
values_df = values_df.rename(columns={"prev_fav": "fav_count"})
values_df.head()

fav_count
word     prev_fav           
clamor   yes               1
clara    no                4
         yes               3
claras   no                3
claridad yes               4

In [641]:
values_df = values_df.reset_index()
values_df.head()

,word,prev_fav,fav_count
0,clamor,yes,1
1,clara,no,4
2,clara,yes,3
3,claras,no,3
4,claridad,yes,4


In [643]:
values_df["fav_num"] = np.where(values_df["prev_fav"] == "yes", values_df["fav_count"], 0)
values_df.head()

,word,prev_fav,fav_count,fav_num
0,clamor,yes,1,1
1,clara,no,4,0
2,clara,yes,3,3
3,claras,no,3,0
4,claridad,yes,4,4


In [644]:
values_df_fav = values_df[["word", "fav_num"]]
values_df_fav.head()

,word,fav_num
0,clamor,1
1,clara,0
2,clara,3
3,claras,0
4,claridad,4


In [646]:
df = df.merge(values_df_fav, how="left", on="word")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,frequency,freq_log_ppm,count,frequency_ppm,hapax,stress,fav_num
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040,6,87.710322,no,stressed,0
1,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,87.710322,4.474040,6,87.710322,no,stressed,1
2,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774,23,336.222901,no,stressed,9
3,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,336.222901,5.817774,23,336.222901,no,stressed,0
4,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,29.236774,3.375427,2,29.236774,no,stressed,0
